# Erstellen eines Dashboards für verantwortungsvolle KI zum Auswerten Ihrer Modelle

Wenn Sie Ihre Machine Learning-Modelle vergleichen und auswerten, sollten Sie mehr als nur ihre Leistungsmetrik überprüfen. Mit Azure Machine Learning können Sie ein Dashboard für verantwortungsvolle KI erstellen, um die Leistung des Modells für verschiedene Kohorten der Daten zu untersuchen. 

## Vorbereiten der Daten

Um das Dashboard für verantwortungsvolle KI zu erstellen, benötigen Sie ein Trainings- und Testdataset, das als Parquet-Dateien gespeichert und als Datenressourcen registriert ist.

Die Daten werden derzeit als CSV-Dateien gespeichert. Wir konvertieren sie in Parquet-Dateien. 


In [ ]:
## Ehe Sie fortfahren

Sie benötigen die neueste Version des Pakets **azureml-ai-ml**, um den Code in diesem Notebook auszuführen. Führen Sie die folgende Zelle aus, um zu überprüfen, ob das Paket installiert ist.

> **Hinweis**:
> Wenn das Paket **azure-ai-ml** nicht installiert ist, führen Sie `pip install azure-ai-ml` aus, um es zu installieren.

In [ ]:
## Herstellen einer Verbindung mit Ihrem Arbeitsbereich

Sie können nun eine Verbindung mit Ihrem Arbeitsbereich herstellen, nachdem Sie die erforderlichen SDK-Pakete installiert haben.

Um eine Verbindung mit einem Arbeitsbereich herzustellen, benötigen Sie Bezeichnerparameter: eine Abonnement-ID, einen Ressourcengruppennamen und einen Arbeitsbereichsnamen. Der Ressourcengruppenname und Arbeitsbereichsname sind bereits für Sie ausgefüllt. Sie müssen lediglich die Abonnement-ID angeben, um den Befehl zu vervollständigen.

Um die erforderlichen Parameter zu finden, klicken Sie rechts oben in Studio auf das Abonnement und den Namen des Arbeitsbereichs. Rechts wird ein Bereich geöffnet.

<p style="color:red;font-size:120%;background-color:yellow;font-weight:bold"> Kopieren Sie die Abonnement-ID, und ersetzen Sie **YOUR-SUBSCRIPTION-ID** durch den Wert, den Sie kopiert haben. </p>

## Erstellen der Datenressourcen

Um das Dashboard für verantwortungsvolle KI zu erstellen, müssen Sie die Trainings- und Testdatasets als **MLtable**-Datenressourcen registrieren. Die MLtable-Datenressourcen verweisen auf die Parquet-Dateien, die Sie zuvor erstellt haben.

In [ ]:
## Erstellen der Pipeline zum Erstellen des Dashboards für verantwortungsvolle KI

Zum Erstellen des Dashboards erstellen Sie eine Pipeline mit vordefinierten Komponenten, die standardmäßig im Azure Machine Learning-Arbeitsbereich registriert sind.

### Registrieren des Modells

Ein Machine Learning-Modell wurde bereits für Sie trainiert. Das Modell sagt voraus, ob ein Patient an Diabetes leidet. Alle Modelldateien werden im Ordner `model` gespeichert. 

Registrieren Sie das Modell, indem Sie auf den Ordner `model` und seinen Inhalt zeigen.

In [ ]:
## Erstellen der Pipeline

Um das Dashboard für verantwortungsvolle KI zu erstellen, erstellen Sie eine Pipeline mithilfe der vordefinierten Komponenten. Sie können auswählen, welche Komponenten verwendet werden sollen und welche Features Sie in Ihr Dashboard aufnehmen möchten. Sie erstellen ein Dashboard, das Fehleranalyse und Modellinterpretierbarkeit umfasst. 

Neben den Features für verantwortungsvolle KI muss eine Pipeline zur Erstellung eines Dashboards am Anfang eine Komponente enthalten, die das Dashboard erstellt, und am Ende eine Komponente, die alle generierten Erkenntnisse sammelt.

In [ ]:
Wenn Sie alle gewünschten Komponenten abgerufen haben, können Sie die Pipeline erstellen und die Komponenten in der richtigen Reihenfolge verbinden:

1. Erstellen Sie das Dashboard.
1. Fügen Sie die Fehleranalyse hinzu.
1. Fügen Sie Erklärungen hinzu.
1. Sammeln Sie alle Erkenntnisse, und visualisieren Sie sie im Dashboard.

Nachdem die Pipeline erstellt wurde, müssen Sie die beiden erforderlichen Eingaben definieren: das Trainings- und das Testdataset.

In [ ]:
Schließlich fügen wir alles zusammen: Weisen Sie der Pipeline die Eingaben zu, und legen Sie die Zielspalte (die vorhergesagte Bezeichnung) fest.

In [ ]:
## Führen Sie die Pipeline aus.

Sobald Sie die Pipeline erfolgreich erstellt haben, können Sie sie übermitteln. Mit dem folgenden Code wird die Pipeline übermittelt und der Status der Pipeline überprüft. Sie können den Status der Pipeline auch in Studio anzeigen. 

Wenn die Pipeline abgeschlossen ist, können Sie das Dashboard in Studio überprüfen. 

In [ ]:
# Get handle to azureml registry for the RAI built in components
registry_name = "azureml"
ml_client_registry = MLClient(
    credential=credential,
    subscription_id=subscription_id,
    resource_group_name=resource_group,
    registry_name=registry_name,
)
print(ml_client_registry)

### Register the model

A machine learning model has already been trained for you. The model predicts whether a patient has diabetes. All model files are stored in the `model` folder. 

Register the model by pointing to the `model` folder and its contents.

In [ ]:
from azure.ai.ml.entities import Model
from azure.ai.ml.constants import AssetTypes

file_model = Model(
    path="model",
    type=AssetTypes.MLFLOW_MODEL,
    name="local-mlflow-diabetes",
    description="Model created from local file.",
)
model = ml_client.models.create_or_update(file_model)

## Build the pipeline

To create the responsible AI dashboard, you'll create a pipeline using the prebuilt components. You can choose which components to use, and which features you want to include in your dashboard. You'll create a dashboard that includes error analysis and model interpretability. 

Next to the responsible AI features, a pipeline to build a dashboard needs to include a component at the start to construct the dashboard, and a component at the end to gather all generated insights.

In [ ]:
model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

In [ ]:
label = "latest"

rai_constructor_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label=label
)

# we get latest version and use the same version for all components
version = rai_constructor_component.version
print("The current version of RAI built-in components is: " + version)

rai_erroranalysis_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=version
)

rai_explanation_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=version
)

rai_gather_component = ml_client_registry.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=version
)

When you've retrieved all components you want to use, you can build the pipeline and connect the components in the appropriate order:

1. Construct the dashboard.
1. Add error analysis.
1. Add explanations.
1. Gather all insights and visualize them in the dashboard.

In [ ]:
from azure.ai.ml import Input, dsl
from azure.ai.ml.constants import AssetTypes

compute_name = "aml-cluster"

@dsl.pipeline(
    compute=compute_name,
    description="RAI insights on diabetes data",
    experiment_name=f"RAI_insights_{model_name}",
)
def rai_decision_pipeline(
    target_column_name, train_data, test_data
):
    # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name,
    )
    create_rai_job.set_limits(timeout=30)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=10)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=10)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=10)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }

Now the pipeline has been built, you need to define the two necessary inputs: the training and test dataset.

In [ ]:
from azure.ai.ml import Input
target_feature = "Diabetic"

diabetes_train_pq = Input(
    type="mltable",
    path=f"azureml:{input_train_data}:{data_version}",
    mode="download",
)
diabetes_test_pq = Input(
    type="mltable",
    path=f"azureml:{input_test_data}:{data_version}",
    mode="download",
)

Finally, we'll put everything together: assign the inputs to the pipeline and set the target column (the predicted label).

In [ ]:
import uuid
from azure.ai.ml import Output

# Pipeline to construct the RAI Insights
insights_pipeline_job = rai_decision_pipeline(
    target_column_name="Diabetic",
    train_data=diabetes_train_pq,
    test_data=diabetes_test_pq,
)

# Workaround to enable the download
rand_path = str(uuid.uuid4())
insights_pipeline_job.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)


## Run the pipeline

When you've successfully built the pipeline, you can submit it. The following code will submit the pipeline and check the status of the pipeline. You can also view the pipeline's status in the studio. 

In [ ]:
from azure.ai.ml.entities import PipelineJob
from IPython.core.display import HTML
from IPython.display import display
import time

def submit_and_wait(ml_client, pipeline_job) -> PipelineJob:
    created_job = ml_client.jobs.create_or_update(pipeline_job)
    assert created_job is not None

    print("Pipeline job can be accessed in the following URL:")
    display(HTML('<a href="{0}">{0}</a>'.format(created_job.studio_url)))

    while created_job.status not in [
        "Completed",
        "Failed",
        "Canceled",
        "NotResponding",
    ]:
        time.sleep(30)
        created_job = ml_client.jobs.get(created_job.name)
        print("Latest status : {0}".format(created_job.status))
    assert created_job.status == "Completed"
    return created_job


# This is the actual submission
insights_job = submit_and_wait(ml_client, insights_pipeline_job)

When the pipeline is completed, you can review the dashboard in the studio. 